# Assignment 5 - Trees and SVMs

# Instructions

This assignment is worth a total of **15 points (10 + 5 bonus)**. The goal of this assignment is to introduce you to decision trees, ensembles of models and SVMs. 

We have structured the assignment into three major parts:

1. **Part One**: Decision Trees,
2. **Part Two**: Support Vector Machines
3. **Part Three**: Revision of all models 

To install any unavailable package, you can do it similarly to the previous assignment.



# Problem 1: Decision Trees, <span style="color:green">total of 5 points </span> 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree

RANDOM_STATE = 40
np.random.seed(RANDOM_STATE)

## Function to plot boundaries of decision trees

In [2]:
def plot_decision_boundary(clf, X, y, axes, cmap):
    x1, x2 = np.meshgrid(
        np.linspace(axes[0], axes[1], 100), np.linspace(axes[2], axes[3], 100)
    )
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)

    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=cmap)
    plt.contour(x1, x2, y_pred, cmap="Greys", alpha=0.8)
    markers = ("o", "^")
    for idx in (0, 1):
        plt.plot(
            X[:, 0][y == idx],
            X[:, 1][y == idx],
            marker=markers[idx],
            linestyle="none",
        )
    plt.axis(axes)


## Sensitivity of Trees to the Orientation of Data

By now, you are hopefully convinced that decision trees offer many advantages: they are relatively easy to understand and interpret, straightforward to use, versatile, and powerful. However, they do come with some limitations. One key drawback is their preference for orthogonal decision boundaries, meaning all splits are perpendicular to an axis. This makes them sensitive to the orientation of the data.

### <span style="color:red">Task 1: Fill out the missing code</span>  <span style="color:green">Total: 2 points </span> 

In [ ]:
# Generate a synthetic dataset
np.random.seed(RANDOM_STATE)
X_square = np.random.rand(100, 2) - 0.5  # Random points in the range [-0.5, 0.5]
y_square = (X_square[:, 0] > 0).astype(np.int64)  # Labels based on x > 0

# Apply a 45-degree rotation
angle = np.pi / 4  # 45 degrees
rotation_matrix = np.array(
    [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
)
X_rotated_square = X_square.dot(rotation_matrix)

# Train a decision tree on the original dataset
tree_clf_square = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=3)
___ # TODO: Fit the decision tree to the original dataset



tree_clf_rotated_square =  # TODO: Create a new DecisionTreeClassifier instance
___ # TODO: Fit the new tree to the rotated dataset


# Visualization
fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(
    tree_clf_square, X_square, y_square, axes=[-0.7, 0.7, -0.7, 0.7], cmap="Pastel1"
)
plt.title("Original Data")
plt.sca(axes[1])
plot_decision_boundary(
    tree_clf_rotated_square,
    X_rotated_square,
    y_square,
    axes=[-0.7, 0.7, -0.7, 0.7],
    cmap="Pastel1",
)
plt.title("Rotated Data")
plt.show()

### <span style="color:red">Task 2: Do PCA first</span>

A potential solution to this issue is to apply a principal component analysis (PCA) transformation. Remember: PCA rotates the data to reduce the correlation between features. This process often (though not always) simplifies things for decision trees.

#### <span style="color:green">Total: 1 point </span> 

In [ ]:
from sklearn.decomposition import PCA

# Generate a synthetic dataset
np.random.seed(RANDOM_STATE)
X_square = np.random.rand(100, 2) - 0.5  # Random points in the range [-0.5, 0.5]
y_square = (X_square[:, 0] > 0).astype(np.int64)  # Labels based on x > 0

# Apply PCA to the dataset
pca = # TODO: Create a PCA instance with 2 components
X_pca =  # TODO: Fit PCA to the dataset and transform the data to the principal component space

# Train a decision tree on the PCA-transformed dataset
tree_clf_pca = DecisionTreeClassifier(random_state=42)
tree_clf_pca.fit(X_pca, y_square)

# Visualization
fig, ax = plt.subplots(figsize=(5, 5))
plot_decision_boundary(
    tree_clf_pca, X_pca, y_square, axes=[-1.5, 1.5, -1.5, 1.5], cmap="Pastel1"
)
plt.title("Decision Tree after PCA transformation")
plt.show()


Now that we've fitted the decision tree, let's bring it to life with a visual representation. We will plot the tree that was generated from the rotated data.

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(
    tree_clf_rotated_square,
    filled=True,
    rounded=True,
)
plt.title("Decision Tree on Rotated Data")
plt.show()

This decision tree shows how the dataset is split based on feature values:

1. **Root Node**:
   - The root node is the starting point. It splits the data into two groups: 54 samples go left (True), and 46 go right (False).
   - The Gini index shows the data is quite mixed at the start.

2. **Splits**:
   - Each node splits the data further based on a feature and a threshold. For example, the second split on the left uses `x[1] <= -0.235`.

3. **Colors**:
   - The node colors represent which class is dominant:
     - **Orange** for one class.
     - **Blue** for the other.
   - Darker colors mean the node is more pure (contains mostly one class).

4. **Leaf Nodes**:
   - The tree ends at leaf nodes, where no further splitting is needed. Most of these nodes have Gini = 0.0, meaning they contain samples from only one class.

## One big tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

np.random.seed(RANDOM_STATE)
X_quad = np.random.rand(200, 1) - 0.5  # a single random input feature
y_quad = X_quad**2 + 0.025 * np.random.randn(200, 1)
tree_reg = DecisionTreeRegressor(max_depth=2, random_state=RANDOM_STATE)
tree_reg.fit(X_quad, y_quad)

### <span style="color:red">Task 3: Which value should you use?</span> 

We want to train a Regression Tree that overfits the data and then visualize it.

<span style="color:green">Total: 2 points </span> 

In [ ]:
tree_reg1 = DecisionTreeRegressor(
    random_state=RANDOM_STATE, max_depth=# TODO: Set max_depth to a value that causes the tree to overfit (e.g., very high depth or no restriction)
)  
tree_reg2 = DecisionTreeRegressor(
    random_state=RANDOM_STATE, max_depth=5, min_samples_leaf=10
)
tree_reg1.fit(X_quad, y_quad)
tree_reg2.fit(X_quad, y_quad)

x1 = np.linspace(-0.5, 0.5, 500).reshape(-1, 1)
y_pred1 = # TODO: Predict values of x1 using tree_reg1
y_pred2 = # TODO: Predict values of x2 using tree_reg1

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

plt.sca(axes[0])
plt.plot(X_quad, y_quad, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([-0.5, 0.5, -0.05, 0.25])
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.legend(loc="upper center")
plt.title("First Tree")

plt.sca(axes[1])
plt.plot(X_quad, y_quad, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([-0.5, 0.5, -0.05, 0.25])
plt.xlabel("$x_1$")
plt.title("Second Tree")


Let's try plotting the tree on the left. It will take some time...

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(
    tree_reg1,
    filled=True,
    rounded=True,
)
plt.title("Decision Tree on Rotated Data")
plt.show()

This decision tree is quite large due to its significant depth, which makes it less interpretable. While deeper trees can capture more complex patterns, they also result in more splits and nodes, making the structure harder to understand.

# Problem 2: Support Vector Machines, <span style="color:red">Task 4: Fill out the code</span> 

In this assignment, we will explore the behavior of Support Vector Machines (SVM) on a synthetic, nonlinear dataset. The goal is to understand how different SVM kernels and hyperparameters (specifically, the regularization parameter C) affect the model's performance, especially in the context of nonlinear decision boundaries.

<span style="color:green">Total of 2 points </span>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Create synthetic nonlinear dataset
X, y = make_circles(n_samples=100, noise=0.1, factor=0.3, random_state=42)

# Step 2: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Step 3: Fit SVM with different kernels and parameters
kernels = ["linear", "poly", "rbf"]
C_values = [0.1, 1, 10]
fig, axes = plt.subplots(len(kernels), len(C_values), figsize=(18, 12))

# Plotting for each kernel and C value
for i, kernel in enumerate(kernels):
    for j, C in enumerate(C_values):
        svm = SVC(kernel=kernel, C=C)
        svm.fit(X_train, y_train)
        y_pred = # TODO: Use the fitted model to make predictions on the test set
        accuracy = # TODO: Compute the accuracy of the model using the test labels and predictions

        ax = axes[i, j]
        ax.set_title(f"Kernel: {kernel}, C={C}, Accuracy: {accuracy:.4f}")

        # Scatter plot of test points
        ax.scatter(
            X_test[:, 0], X_test[:, 1], c=y_test, cmap="coolwarm", edgecolors="k", s=50
        )

        # Plot decision boundary
        h = 0.02
        x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
        y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        # Contour plot of decision boundary
        ax.contourf(xx, yy, Z, alpha=0.8)
        ax.set_xlim(X_test[:, 0].min() - 1, X_test[:, 0].max() + 1)
        ax.set_ylim(X_test[:, 1].min() - 1, X_test[:, 1].max() + 1)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


# Problem 3: Real world scenario - Model Comparison Using Multiple Classifiers, <span style="color:green">total of 8 points </span> 

In this scenario, we are tasked with evaluating multiple classification models on several datasets of varying complexities. These datasets include well-known ones like Iris and Wine, as well as more challenging ones like the Moon and a hard synthetic dataset.

### Process Overview
Typically, when working on a classification problem, I would follow a structured approach to model selection and tuning:

1. **Dataset Preprocesing**: Analyze the characteristics of each dataset, such as the number of features, the distribution of target classes, and the complexity of the relationships between the features. Data preprocessing usually takes the most "human" time.
2. **Model Selection**: Try different types of models, such as K-Nearest Neighbors (KNN), Logistic Regression, Decision Trees, Random Forests, and Support Vector Machines (SVM). Each model has its strengths and weaknesses depending on the nature of the data. Usually we try the default parameters of each algorithm, and then select a subset of them to hyperparameter tune.
3. **Hyperparameter Tuning**: Use techniques like Grid Search to tune the hyperparameters of each model. This process involves selecting the best combination of parameters (such as the number of neighbors in KNN, the regularization strength in Logistic Regression, or the depth of the trees in Decision Trees).
4. **Model Evaluation**: Evaluate each model using testing metrics like accuracy, precision, recall, and F1 score. Choose the model that performs best on the test data.


### Approach in our case

1. **Try Different Models**: We evaluate several models, including KNN, Logistic Regression, Decision Trees, Random Forests, and SVMs, to understand how they perform on each dataset.
2. **Hyperparameter Tuning**: We use Grid Search to find the best hyperparameters for each model, adjusting values such as the number of neighbors for KNN, regularization strength for Logistic Regression, and tree depth for Decision Trees.
3. **Evaluation**: We measure model performance using accuracy, as well as training time, to ensure efficient results.

### Conclusion

By applying this approach across different datasets, we can determine which model is best suited for each case. This method is effective for exploring various classifiers and tuning them to achieve optimal performance.

### <span style="color:red">Task 5: Fill out the missing code - 7 points</span>

In [ ]:
import time
from sklearn.datasets import (
    load_iris,
    load_wine,
    load_digits,
    make_moons,
    make_classification,
)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

# Step 1: Load datasets
datasets = {
    "Iris": load_iris(return_X_y=True),
    "Wine": load_wine(return_X_y=True),
    "Digits": load_digits(return_X_y=True),
    "Moon": make_moons(n_samples=1000, noise=0.3, random_state=42),  # Moon dataset
    "Hard Synthetic": make_classification(  # Hard synthetic dataset
        n_samples=1000,
        n_features=50,
        n_informative=25,
        n_redundant=15,
        n_classes=5,
        n_clusters_per_class=2,
        random_state=42,
        flip_y=0.3,
        class_sep=2,
        weights=[0.2, 0.2, 0.2, 0.2, 0.2],
    ),
}

# Step 2: Define classifiers and hyperparameter grids
classifiers = {
    "KNN": #TODO: Define the KNN classifier
    "Logistic Regression": # TODO: Define the Logistic Regression classifier
    "Decision Tree": # TODO: Define the Decision Tree classifier
    "Random Forest": RandomForestClassifier(n_jobs=-1),  
    "SVM":  # TODO: Define Support Vector Machine classifier
}

param_grids = {
    "KNN": {"n_neighbors": [5, 7]},
    "Logistic Regression": {"C": [0.1, 1]},
    "Decision Tree": {"max_depth": [3, 5]},
    "Random Forest": {"n_estimators": [50], "max_depth": [3]},
    "SVM": {"C": [0.1, 1], "kernel": ["linear"]},
}

# Step 3: Initialize results storage
overall_results = {}

# Step 4: Process each dataset
for dataset_name, (X, y) in datasets.items():
    print(f"\nProcessing dataset: {dataset_name}")
    # Step 4.1: Split the dataset
    X_train, X_test, y_train, y_test = # TODO: split the data in train/test. Set test_size to 0.3

    # Step 4.2: Fit models and perform GridSearchCV
    results = {}
    model_performance = {
        model_name: [] for model_name in classifiers.keys()
    }  # Track performance
    model_times = {
        model_name: [] for model_name in classifiers.keys()
    }  # Track training times

    for model_name, model in classifiers.items():
        # Record the start time of GridSearchCV
        start_time = time.time()

        
        grid_search = # TODO: Use GridSearchCV for hyperparameter tuning. Use 3-fold cross validation
        ____  # TODO: Fit the model using training data


        # Record the end time of GridSearchCV
        end_time = time.time()
        training_time = end_time - start_time  # Time taken for the entire grid search

        best_model = # TODO: Get the best model from GridSearchCV
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Store the results
        results[model_name] = {
            "Best Params": # TODO: Get the parameters of the best model from GridSearchCV
            "Accuracy": accuracy,
            "Training Time (s)": training_time,
        }

        # Append performance and training time
        model_performance[model_name].append(accuracy)
        model_times[model_name].append(training_time)

    # Step 4.3: Print results for the current dataset
    print(f"Results for {dataset_name} Dataset:")
    for model_name, result in results.items():
        print(
            f"  {model_name} - Best Params: {result['Best Params']} | Accuracy: {result['Accuracy']:.4f} | "
            f"Training Time: {result['Training Time (s)']:.4f} seconds"
        )
    print("\n" + "-" * 50 + "\n")

    # Step 4.4: Store overall results
    overall_results[dataset_name] = results

# Step 5: Print average performance across datasets
print("Average Performance and Training Time per Model Across All Datasets:")
model_avg_performance = {}
model_avg_times = {}

for model_name in classifiers.keys():
    all_accuracies = []
    all_times = []
    for dataset_name, dataset_results in overall_results.items():
        all_accuracies.append(dataset_results[model_name]["Accuracy"])
        all_times.append(dataset_results[model_name]["Training Time (s)"])
    model_avg_performance[model_name] = # TODO: Compute average accuracy
    model_avg_times[model_name] = # TODO: Compute average training time

    print(
        f"{model_name}: Average Accuracy = {model_avg_performance[model_name]:.4f} | "
        f"Average Training Time = {model_avg_times[model_name]:.4f} seconds"
    )


### Which Model Seems to Be the Best for Each Dataset?  <span style="color:red">Write down the list of best models - 1 point</span>


After evaluating multiple models (KNN, Logistic Regression, Decision Trees, Random Forests, and SVMs) across several datasets, we can summarize the best-performing models for each dataset based on accuracy and training time. Write down which models achieved the highest accuracy. If there are multiple models, write them all down.

#### **Iris Dataset**
- **Best Model**: 

#### **Wine Dataset**
- **Best Model**: 

#### **Digits Dataset**
- **Best Model**: 

#### **Moon Dataset**
- **Best Model**: 

#### **Synthetic Dataset**
- **Best Model**: 


<span style="color:#4CAF50">Congratulations on reaching the end of this assignment! Well done for completing all the assignments. Best of luck with your exams – keep up the great work and stay confident!</span>